In [16]:
# coding: utf-8
import sys
import re
import random
from operator import itemgetter
import urlparse
from posixpath import split, splitext, normpath
from collections import defaultdict, Counter
from random import sample
from itertools import chain
import pandas as pd
from sklearn.cluster import DBSCAN

In [37]:
numeric_substr = re.compile('[^\d]+\d+[^\d]+$')


def is_number(s):
    try:
        int(s)
        return True
    except ValueError:
        return False


def read_lines(file_name):
    with open(file_name) as f:
        lines = f.readlines()
        lines = [urlparse.unquote(x.strip()) for x in lines]
        #lines = [x.strip() for x in lines]
    return lines

def write_lines(file_name, url_list):
    with open(file_name, 'w') as f:
        lines = '\n'.join(url_list)
        f.write(lines)

def path_to_segments(path):
    tmp = path
    tmp = normpath(tmp)
    segments = []
    while tmp != "/":
        (tmp, last) = split(tmp)
        segments.insert(0, last)
    return segments


def segments_to_features(segments):
    features = []

    # 1
    features.append('segments:' + str(len(segments)))

    for index, string in enumerate(segments):
        # 4a
        features.append('segment_name_' + str(index) + ':' + str(string))
        
        # 4d
        extension = splitext(string)[1][1:]
        if extension:
            features.append('segment_ext_' + str(index) + ':' + extension)
        
        # 4b
        if is_number(string):
            features.append('segment_[0-9]_' + str(index) + ':' + '1')
        else:
           # 4c
            string = urlparse.unquote(string)
  
            if numeric_substr.match(string):
                features.append('segment_substr[0-9]_' + str(index) + ':' + '1')
                # 4e
                if extension:
                    features.append(
                        'segment_ext_substr[0-9]_' + str(index) + ':' + extension)
        
        # 4f
        features.append('segment_len_' + str(index) + ':' + str(len(string)))      
        
    return features

def query_to_params_features(query):
    features = []
    if query:        
        parameters = dict(urlparse.parse_qsl(query))
        # 2
        features.append('param_name:' + str(parameters.keys()))
        for parameter, value in parameters.iteritems():
            # 3
            features.append('param:' + str(parameter) + '=' + str(value))   
    return features

def url_to_features_list(url):
    features_list = []
    parsed_url = urlparse.urlparse(url)
    segments = path_to_segments(parsed_url.path)
    features_list += segments_to_features(segments)
    features_list += query_to_params_features(parsed_url.query)
    return features_list

def url_list_to_features(url_list):
    features = defaultdict(list)
    for url in url_list:
        features[url] = url_to_features_list(url)
    return features

def url_list_to_all_features(url_list):
    features = []
    for url in url_list:
        parsed_url = urlparse.urlparse(url)
        segments = path_to_segments(parsed_url.path)
        features += segments_to_features(segments)
        features += query_to_params_features(parsed_url.query)
    return features
        
    
def choose_features(dictionary_of_lists, threshold):
    frequencies = Counter(chain.from_iterable(dictionary_of_lists.values()))
    frequencies = {feature:count for feature, count in frequencies.iteritems() if count > threshold}
    return frequencies.keys()

def write_features(file_name, features):
    with open(file_name, 'w') as f:
        f.write('\n'.join('%s\t%s' % x for x in features))
       
def extract_features(INPUT_FILE_1, INPUT_FILE_2, OUTPUT_FILE):
    
    examined_urls = read_lines(INPUT_FILE_1)
    general_urls = read_lines(INPUT_FILE_2)\
    
    size = len(examined_urls) / 2
    examined_urls = sample(examined_urls, size)
    general_urls = sample(general_urls, size)
    
    # features = url_list_to_features(examined_urls + general_urls)    
    features = url_list_to_all_features(examined_urls + general_urls) 
    
    # frequency
    alpha = 0.1
    threshold = size * alpha
    # write_features(OUTPUT_FILE, frequency(features, threshold))
    frequencies = Counter(features)
    frequencies = {feature:count for feature, count in frequencies.iteritems() if count > threshold}
    frequencies = sorted(frequencies.items(), key=itemgetter(1), reverse=True)
    write_features(OUTPUT_FILE, frequencies)
    # return features
    # print >> sys.stderr, "Not implemented"




In [1]:
import sys
import os
import re
import random
import time
#from sklearn.cluster import <any cluster algorithm>
import numpy as np

In [39]:
df2 = pd.DataFrame(np.random.randint(low=0, high=10, size=(5, 5)), columns=['a', 'b', 'c', 'd', 'e'])

In [40]:
type(df2.columns)

pandas.core.indexes.base.Index

In [41]:
examined_urls = read_lines('data/urls.wiki.examined')
general_urls = read_lines('data/urls.wiki.general')
    
size = 500
examined_urls = sample(examined_urls, size)
general_urls = sample(general_urls, size)

IOError: [Errno 2] No such file or directory: 'data/urls.wiki.examined'

In [ ]:
class sekitei: pass

def dictionary_of_lists_to_dataframe(dictionary_of_lists):
    df = pd.DataFrame.from_dict(dictionary_of_lists, orient='index')
    df = df.stack().reset_index()
    df.level_1 = 1
    df = df.pivot(index='level_0',columns=0,values='level_1').fillna(0)
    return df

def combine_features(qlink_features, unknown_features):
    qlink_df = dictionary_of_lists_to_dataframe(qlink_features)
    qlink_df['is_qlink'] = 1
    unknown_df = dictionary_of_lists_to_dataframe(unknown_features)
    unknown_df['is_qlink'] = 0
    df = pd.concat([qlink_df, unknown_df], axis=0).fillna(0)
    # !!! contains is_qlink info
    return df

def define_segments(QLINK_URLS, UNKNOWN_URLS, QUOTA):
    global sekitei
    #print "define_segments is not implemented";
    qlink_features = url_list_to_features(QLINK_URLS)
    unknown_features = url_list_to_features(UNKNOWN_URLS)
    #all_features_dict = merge_two_dicts(qlink_features, unknown_features)
    #alpha = 
    #size =
    #threshold_frequency = size * alpha
    #sekitei.dataframe = sekitei.dataframe[choose_features(all_features_dict, threshold_frequency)]
    sekitei.dataframe = combine_features(qlink_features, unknown_features)
    train = sekitei.dataframe.drop(['is_qlink'], axis=1).values
    number_of_clusters = 10
    sekitei.kmeans = KMeans(n_clusters=number_of_clusters).fit(train)
    sekitei.dataframe['label'] = sekitei.kmeans.labels_
    
    



In [ ]:
#global sekitei
define_segments(examined_urls, general_urls, 500)

In [ ]:
sekitei.dataframe

In [ ]:
#
# returns True if need to fetch url
#
def fetch_url(url):
    #return sekitei.fetch_url(url);
    current_url_features = url_to_features_list(url)
    current_url_row = np.isin(sekitei.dataframe.drop(['is_qlink', 'label'], axis=1).columns, current_url_features)
    current_url_label = sekitei.kmeans.predict(current_url_row.reshape(1, -1))[0]
    print current_url_label
    return True;


In [ ]:
(sekitei.dataframe[['label', 'is_qlink']].groupby('label').mean() * 10)['is_qlink'].loc[0]

In [42]:
pd.Series(index=(sekitei.dataframe[['label', 'is_qlink']].groupby('label').mean() * 10)['is_qlink'] .index, dtype=int64)

NameError: name 'int64' is not defined

In [43]:
defaultdict.fromkeys(['a','b','c'], 0)

defaultdict(None, {'a': 0, 'b': 0, 'c': 0})

In [44]:
fetch_url('http://lenta.ru/articles/2003/05/29/everest/')

8


True

In [45]:
current_url_features = url_to_features_list('http://lenta.ru/articles/2003/05/29/everest/')
current_url_features

['segments:5',
 'segment_name_0:articles',
 'segment_len_0:8',
 'segment_name_1:2003',
 'segment_[0-9]_1:1',
 'segment_len_1:4',
 'segment_name_2:05',
 'segment_[0-9]_2:1',
 'segment_len_2:2',
 'segment_name_3:29',
 'segment_[0-9]_3:1',
 'segment_len_3:2',
 'segment_name_4:everest',
 'segment_len_4:7']

In [46]:
current_url_row = np.isin(sekitei.dataframe.drop(['is_qlink', 'label'], axis=1).columns, current_url_features)

In [47]:
current_url_label = sekitei.kmeans.predict(current_url_row.reshape(1, -1))[0]

In [48]:
current_url_label

8

In [83]:
wiki = read_lines('data/urls.wikipedia.examined')

In [84]:
for i in  wiki:
    print i

http://ru.wikipedia.org/wiki/Дакар_(подводная_лодка)
http://ru.wikipedia.org/wiki/Присно_2
http://ru.wikipedia.org/wiki/Сийи-ла-Потри
http://ru.wikipedia.org/wiki/������������_�����������_�_�������-�����
http://ru.wikipedia.org/wiki/Вик-ауф-Фёр
http://ru.wikipedia.org/wiki/Ваерланд,_Аре
http://ru.wikipedia.org/wiki/Медухово
http://ru.wikipedia.org/wiki/Flying_Elephant
http://ru.wikipedia.org/wiki/Суета_вокруг_колец_возвращается:_По-королевски
http://ru.wikipedia.org/wiki/Противопульные_вставки_SAPI_и_ESAPI_бронежилетов_сухопутных_войск
http://ru.wikipedia.org/wiki/Легитимность?previous=yes
http://ru.wikipedia.org/wiki/��4�
http://ru.wikipedia.org/wiki/Список_австралийских_штатов_и_территорий_по_численности_населения
http://ru.wikipedia.org/wiki/Богачево_(Баймакский_район)
http://ru.wikipedia.org/wiki/Файл:Nordwind_Airlines_Boeing_757-200_VQ-BBT_SVO_2009-1-2.png
http://ru.wikipedia.org/wiki/Категория:Красноармейский_район_Приморского_края
http://ru.wikipedia.org/wiki/����
http://ru.wiki

http://ru.wikipedia.org/wiki/Kepler-12b
http://ru.wikipedia.org/wiki/������_��������
http://ru.wikipedia.org/wiki/Рир-экран
http://ru.wikipedia.org/wiki/No-man
http://ru.wikipedia.org/wiki/Ватанабэ,_Хироси
http://ru.wikipedia.org/wiki/��������,_�������_����������
http://ru.wikipedia.org/wiki/������
http://ru.wikipedia.org/wiki/�����
http://ru.wikipedia.org/wiki/Файл:Pride_&_Prejudice_1940.jpg
http://ru.wikipedia.org/wiki/�����������_(�����,_1992)
http://ru.wikipedia.org/wiki/Коды_Тейлора
http://ru.wikipedia.org/wiki/�������
http://ru.wikipedia.org/wiki/Стефан_Лакапин
http://ru.wikipedia.org/wiki/Шаблон:Математические_знаки
http://ru.wikipedia.org/wiki/Файл:LIHACH.jpg
http://ru.wikipedia.org/wiki/���������
http://ru.wikipedia.org/wiki/��������_(�������)
http://ru.wikipedia.org/wiki/Категория:Родившиеся_в_Черновицкой_области
http://ru.wikipedia.org/wiki/�����_�����_����������_����_1992
http://ru.wikipedia.org/wiki/Ip_address
http://ru.wikipedia.org/wiki/Летучая_белка
http://ru.wikipedia.

In [85]:
wiki = read_lines('data/urls.wikipedia.general')

In [86]:
for i in  wiki:
    print i

http://ru.wikipedia.org/wiki/Санта-Комба_(значения)
http://ru.wikipedia.org/wiki/Obotin
http://ru.wikipedia.org/wiki/Тальменский_район_Алтайского края
http://ru.wikipedia.org/wiki/Кнеф,_Хильдегард
http://ru.wikipedia.org/wiki/NGC 4606
http://ru.wikipedia.org/wiki/Tonya D. Clayton
http://ru.wikipedia.org/wiki/Чами,_Альфонс
http://ru.wikipedia.org/wiki/Корчак_Януш
http://ru.wikipedia.org/wiki/Мухина Надежда
http://ru.wikipedia.org/wiki/Карамай_(Костанайская_область)
http://ru.wikipedia.org/wiki/Файл:Grabin.jpg
http://ru.wikipedia.org/wiki/1467
http://ru.wikipedia.org/wiki/Джидеш_Ногойбаевна_Истамбекова
http://ru.wikipedia.org/wiki/16468081_(число)
http://ru.wikipedia.org/wiki/Файл:Stolbov_anatoliy.jpg
http://ru.wikipedia.org/wiki/Надежда  Евдокимова
http://ru.wikipedia.org/wiki/Обсуждение:Аракс_(космический_аппарат)
http://ru.wikipedia.org/wiki/Левек, Шарль
http://ru.wikipedia.org/wiki/649_до_н._э.
http://ru.wikipedia.org/wiki/Ð¡Ð»ÑÐ¶ÐµÐ±Ð½Ð°Ñ:Search/ÐÒ¯Ð½ÑÑÐ°ÑÐ¾Ð²Ð°_Ð.
http://ru.

http://ru.wikipedia.org/wiki/%D0%9C%D0%B0%D0%B3%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D%200%B8%D0%B9_%D1%80%D0%B5%D0%B0%D0%BB%D0%B8%D0%B7%D0%20%BC
http://ru.wikipedia.org/wiki/Лугаль-далу?previous=yes
http://ru.wikipedia.org/wiki/Файл:Wappen_von_Schwienau.png
http://ru.wikipedia.org/wiki/�������,_�����_���������
http://ru.wikipedia.org/wiki/Гусев,_Евгений
http://ru.wikipedia.org/wiki/Элик Мазитович Халимов
http://ru.wikipedia.org/wiki/Мук, Карл
http://ru.wikipedia.org/wiki/Robert N. Rosen
http://ru.wikipedia.org/wiki/Файл:Khanty-Mansi_in_Russia.svg
http://ru.wikipedia.org/wiki/Гудбурово
http://ru.wikipedia.org/wiki/Служебная:Ссылки_сюда/Портал:Пермь/В_этот_день/5_марта
http://ru.wikipedia.org/wiki/Тионетский_уезд
http://ru.wikipedia.org/wiki/Файл:Wappen_Wilsum.svg
http://ru.wikipedia.org/wiki/Категория:Персоналии:Тыхы
http://ru.wikipedia.org/wiki/Категория:Умершие_в_Мехабаде
http://ru.wikipedia.org/wiki/Гран-при_Кореи_2011_года
http://ru.wikipedia.org/wiki/А._Ф._Баландин
http://ru.wikipedia.or

http://ru.wikipedia.org/wiki/Весовые ограничения
http://ru.wikipedia.org/wiki/Квантовый_переход
http://ru.wikipedia.org/wiki/Файл:Aurelie_REVILLET-4bf58.jpg
http://ru.wikipedia.org/wiki/Лина_Яковлевна_Каменская
http://ru.wikipedia.org/wiki/Евгений Павлович Каргаполов
http://ru.wikipedia.org/wiki/Инкубатор:Чо_Мен_Хи
http://ru.wikipedia.org/wiki/Категория:Палеонтологи_по_алфавиту
http://ru.wikipedia.org/wiki/Служебная:Связанные_правки/Проект:Удмуртия/НП/Карта_Алнашского_района
http://ru.wikipedia.org/wiki/«Che_Francisco»
http://ru.wikipedia.org/wiki/скачать_fifa_manager_2009
http://ru.wikipedia.org/wiki/Категория:Сезоны_футбольных_клубов_1899/1900
http://ru.wikipedia.org/wiki/Файл:V_zvaigzde_2.png
http://ru.wikipedia.org/wiki/Экс_(остров)?previous=yes
http://ru.wikipedia.org/wiki/Обсуждение:Майлендер,_Бернд
http://ru.wikipedia.org/wiki/Эрих_Борисович_Голланд
http://ru.wikipedia.org/wiki/Manga_Calendar
http://ru.wikipedia.org/wiki/11045249_(число)
http://ru.wikipedia.org/wiki/Обсуждение:К

http://ru.wikipedia.org/wiki/Рав-самаль_миткадем
http://ru.wikipedia.org/wiki/Файл:Dinitronaphthalene_composition.png
http://ru.wikipedia.org/wiki/Ю._Стрельцов
http://ru.wikipedia.org/wiki/Файл:Williams_FW07_at_Barber_02.jpg
http://ru.wikipedia.org/wiki/Русский_драматический_театр_им._А._П._Чехова
http://ru.wikipedia.org/wiki/14602699_(число)
http://ru.wikipedia.org/Служебная:Log?page=Экзорцизм
http://ru.wikipedia.org/wiki/Файл:Porsche_597_1953_frontleft_2009-03-14_A.JPG
http://ru.wikipedia.org/wiki/Шаблон:ПересадкаНаМетроСПБ
http://ru.wikipedia.org/wiki/user:80.254.110.6
http://ru.wikipedia.org/wiki/Сахалинский международный театральный центр имени А.П. Чехова
http://ru.wikipedia.org/wiki/Файл:On_the_night.jpg
http://ru.wikipedia.org/wiki/Скачать_разные_проги_и_команды_для_кс
http://ru.wikipedia.org/wiki/Ð¤Ð°Ð¹Ð»:Fire-animation.gif
http://ru.wikipedia.org/wiki/12445780_(число)
http://ru.wikipedia.org/wiki/Лик (мифология)
http://ru.wikipedia.org/wiki/Категория:Заслуженные_деятели_науки

http://ru.wikipedia.org/wiki/12446989_(число)
http://ru.wikipedia.org/wiki/Обсуждение:Кузеево_(Шавьядинский_сельсовет)
http://ru.wikipedia.org/wiki/Yin
http://ru.wikipedia.org/wiki/Скачать_русские_рукописные_шрифты_на_фотошоп
http://ru.wikipedia.org/wiki/Суртанды
http://ru.wikipedia.org/wiki/Barclays_Dubai_Tennis_Championships_2010_-_Женский_парный_турнир
http://ru.wikipedia.org/wiki/Башкирские_восстания
http://ru.wikipedia.org/wiki/Lucenzo feat. Big Ali
http://ru.wikipedia.org/wiki/Пачоли_Лука?previous=yes
http://ru.wikipedia.org/wiki/Жулио Сезар Соарес Эспиндола
http://ru.wikipedia.org/wiki/Ингридиент
http://ru.wikipedia.org/wiki/Ахмед Устарханов
http://ru.wikipedia.org/wiki/Файл:Gymnocalicium_mihanowichii,_Ciudad_Ho_Chi_Minh,_Vietnam,_2013-08-14,_DD_02.JPG
http://ru.wikipedia.org/wiki/NGC 1079
http://ru.wikipedia.org/wiki/Обсуждение:Так,_Джессика
http://ru.wikipedia.org/wiki/Фисташка_мастиковая
http://ru.wikipedia.org/wiki/Камчатская_флотилия_ВМФ_России
http://ru.wikipedia.org/wiki/

...

http://ru.wikipedia.org/wiki/Категория:Футбольные_тренеры_Индии
http://ru.wikipedia.org/wiki/Вестрис,_Тереза
http://ru.wikipedia.org/wiki/Файл:Bow_Cello.jpg
http://ru.wikipedia.org/wiki/Морской транспорт
http://ru.wikipedia.org/wiki/Павел (митрополит Нижегородский)
http://ru.wikipedia.org/wiki/Крупицкий,_Евгений_Михайло<a href='http:/www.nedug.ru/desease/вич'>вич</a>
http://ru.wikipedia.org/wiki/Задача_о_самолете
http://ru.wikipedia.org/wiki/Хованский,_Николай
http://ru.wikipedia.org/wiki/Дзета_Зайца
http://ru.wikipedia.org/wiki/Файл:Scott_Steiner_2013.jpg
http://ru.wikipedia.org/wiki/Прибрежный_(Амурская_область)
http://ru.wikipedia.org/wiki/Шаблон:РисПортала/Новейшее_время
http://ru.wikipedia.org/wiki/11946134_(число)
http://ru.wikipedia.org/wiki/User:BendelacBOT
http://ru.wikipedia.org/wiki/Джон Фрум
http://ru.wikipedia.org/wiki/Якир_Яков_Иосифович
http://ru.wikipedia.org/wiki/Файл:Panas_Myrnyj.JPG
http://ru.wikipedia.org/wiki/Havana Brown feat. R3hab & Prophet
http://ru.wikipedia.o

http://ru.wikipedia.org/wiki/Герб_Баппагайинского_наслега
http://ru.wikipedia.org/wiki/Центральный_
http://ru.wikipedia.org/wiki/Обсуждение:Пацифизм
http://ru.wikipedia.org/wiki/Jutty_Ranx
http://ru.wikipedia.org/wiki/Файл:Escudo_de_Tornavacas.svg
http://ru.wikipedia.org/wiki/Маргарита_Геннадьевна_Кондырева
http://ru.wikipedia.org/wiki/1258_год_до_н._э.
http://ru.wikipedia.org/wiki/Сусько, Яков Егорович
http://ru.wikipedia.org/wiki/Файл:Isabelle_de_Charrière_-_Quentin_de_La_Tour.jpg
http://ru.wikipedia.org/wiki/Хитченс,_Роберт?previous=yes
http://ru.wikipedia.org/wiki/Кларк, Анна
http://ru.wikipedia.org/wiki/12871707_(число)
http://ru.wikipedia.org/wiki/Файл:MinasGerais_Municip_Cristais.svg
http://ru.wikipedia.org/wiki/Breda Ba.65
http://ru.wikipedia.org/wiki/Мезотич, Даниэль
http://ru.wikipedia.org/wiki/Дитрих_Абек
http://ru.wikipedia.org/wiki/Средняя_общеобразовательная_школа_№9_(Обнинск)
http://ru.wikipedia.org/wiki/Кёкусинкай_каратэ
http://ru.wikipedia.org/wiki/Краснофлотский
http:

http://ru.wikipedia.org/wiki/Эрнесту_Гейзель
http://ru.wikipedia.org/wiki/Ягодненский сельский совет Харьковской области
http://ru.wikipedia.org/wiki/Лоуренсу,_Эдуарду?previous=yes
http://ru.wikipedia.org/wiki/Планеты_мира_<Кин-дза-дза!>
http://ru.wikipedia.org/wiki/user:82.131.29.176
http://ru.wikipedia.org/wiki/Portlethen
http://ru.wikipedia.org/wiki/ru:Усть-Ишим?uselang=ru
http://ru.wikipedia.org/wiki/Леонид Васильевич Деев
http://ru.wikipedia.org/wiki/14746270_(число)
http://ru.wikipedia.org/wiki/Проект:Знаете_ли_вы/Архив_рубрики/2006-08?previous=yes
http://ru.wikipedia.org/wiki/Purity?previous=yes
http://ru.wikipedia.org/wiki/Mari_Ferrari_feat._Dhany
http://ru.wikipedia.org/wiki/Алексеев,_Андрей_Корнеевич
http://ru.wikipedia.org/wiki/Файл:Charles_Montesquieu.jpg
http://ru.wikipedia.org/wiki/16428091_(число)
http://ru.wikipedia.org/wiki/16212460_(число)
http://ru.wikipedia.org/wiki/15360833_(число)
http://ru.wikipedia.org/wiki/Шаблон:Медаль_80_лет_Госкомспорту_России
http://ru.wiki

http://ru.wikipedia.org/wiki/Файл:Niameyairport_2005_crop.JPG
http://ru.wikipedia.org/wiki/Изображение:EKGI.png
http://ru.wikipedia.org/wiki/КХЛ_в_сезоне_2011/2012
http://ru.wikipedia.org/wiki/Файл:King_Songsten_Gampo's_statue_in_his_meditation_cave_at_Yerpa.jpg
http://ru.wikipedia.org/wiki/Гелий Иванович Мосеев
http://ru.wikipedia.org/wiki/10684506_(число)
http://ru.wikipedia.org/wiki/Епархия Тампико
http://ru.wikipedia.org/wiki/CAPTCHA?useskin=custis
http://ru.wikipedia.org/wiki/Георгий I (царь Грузии)
http://ru.wikipedia.org/wiki/User_talk:AXP16
http://ru.wikipedia.org/wiki/Регистрация в одноклассниках сейчас
http://ru.wikipedia.org/wiki/Собор Экса
http://ru.wikipedia.org/wiki/Герасимов Александр Михайлович
http://ru.wikipedia.org/wiki/Обсуждение:Кукес
http://ru.wikipedia.org/wiki/Служебная:History/Проект:Белгородская_область/Участники
http://ru.wikipedia.org/wiki/Ян_I
http://ru.wikipedia.org/wiki/Файл:Gloria_Gaynor_2003.jpg
http://ru.wikipedia.org/wiki/Laurie Zuckerman
http://ru.wi

http://ru.wikipedia.org/wiki/User:Aleksandr_Kozlovskij
http://ru.wikipedia.org/wiki/С._И._Сполитак
http://ru.wikipedia.org/wiki/Калиновка_(Бижбулякский_район)
http://ru.wikipedia.org/wiki/Категория:Кэлерова_геометрия
http://ru.wikipedia.org/wiki/Файл:Bundesarchiv_Bild_101I-708-0300-03,_Russland-Süd,_Panzer_IV_in_Fahrt.jpg
http://ru.wikipedia.org/wiki/Чемпионат СССР по хоккею с шайбой 1984/1985
http://ru.wikipedia.org/wiki/Сионе_Тому
http://ru.wikipedia.org/wiki/Категория:Ponte_25_de_Abril
http://ru.wikipedia.org/wiki/Иван_Тимофеевич_Черных
http://ru.wikipedia.org/wiki/Паттон_Джордж_Смит
http://ru.wikipedia.org/wiki/Михаил Поздняев
http://ru.wikipedia.org/wiki/Файл:Red_Army_Insignia_2b.svg
http://ru.wikipedia.org/wiki/Древнеисландский
http://ru.wikipedia.org/wiki/ArcWeb_Services?previous=yes
http://ru.wikipedia.org/wiki/Потоцкий,_Юзеф_Фелициан
http://ru.wikipedia.org/wiki/Файл:Jerome_bonaparte.jpg
http://ru.wikipedia.org/wiki/Эмфизематозный карбункул
http://ru.wikipedia.org/U-20_(1912)


In [51]:
path = urlparse.urlparse('http://ru.wikipedia.org/wiki/Чернозубов,_Илья_Фёдорович').path

In [52]:
print path

/wiki/Чернозубов,_Илья_Фёдорович


In [53]:
surname = re.compile('^/wiki/[^/]+,_[^/]+$')
if surname.match(path):
        print 'yeah'

yeah


In [54]:
regex = re.compile('^/wiki/Файл:[^/]+$')

In [55]:
if regex.match(path):
    print 'ok'

In [56]:
wiki_features = {'category:1': re.compile('^/wiki/Категория:[^/]+$'),
'wiki:1' : re.compile('^/wiki/Википедия:[^/]+$'),
'portal:1' : re.compile('^/wiki/Портал:[^/]+$'),
'project:1' : re.compile('^/wiki/Проект:[^/]+$'),
'template:1' : re.compile('^/wiki/Шаблон:[^/]+$'),
'service:1' : re.compile('^/wiki/Служебная:[^/]+$'),
'file_page:1' : re.compile('^/wiki/Файл:[^/]+$'),
'talk:1' : re.compile('^/wiki/Обсуждение:[^/]+$'),
'image:1' : re.compile('^/wiki/Изображение:[^/]+$'),
'incubator:1' : re.compile('^/wiki/Инкубатор:[^/]+$')}

In [57]:
def path_to_wiki_features(path):
    features = []
    for name, regex in wiki_features.iteritems():
        if regex.match(path):
            features.append(name)
    return features

In [58]:
path_to_wiki_features(path)

[]

In [59]:
help(re)

Help on module re:

NAME
    re - Support for regular expressions (RE).

FILE
    /usr/lib/python2.7/re.py

MODULE DOCS
    http://docs.python.org/library/re

DESCRIPTION
    This module provides regular expression matching operations similar to
    those found in Perl.  It supports both 8-bit and Unicode strings; both
    the pattern and the strings being processed can contain null bytes and
    characters outside the US ASCII range.
    
    Regular expressions can contain both special and ordinary characters.
    Most ordinary characters, like "A", "a", or "0", are the simplest
    regular expressions; they simply match themselves.  You can
    concatenate ordinary characters, so last matches the string 'last'.
    
    The special characters are:
        "."      Matches any character except a newline.
        "^"      Matches the start of the string.
        "$"      Matches the end of the string or just before the newline at
                 the end of the string.
        "*"    

In [60]:
wiki = re.compile('^http://ru.wikipedia.org/wiki/[.]+')
if wiki.match(sekitei.dataframe.index[0]):
    write_lines('urls', list(sekitei.dataframe.index))

In [61]:
sekitei.dataframe.index[0]

'http://lenta.ru/articles/2005/07/12/uhta/'

In [62]:
read = read_lines('unknown')

In [64]:
for i in read:
    print i

http://ru.wikipedia.org/wiki/�����������,_���_�����������_�������_���������_(�����)
http://ru.wikipedia.org/wiki/��������,_��������_����������
http://ru.wikipedia.org/wiki/������������_������_����������
http://ru.wikipedia.org/wiki/���������,_����������_�������
http://ru.wikipedia.org/wiki/���������_�����
http://ru.wikipedia.org/wiki/������
http://ru.wikipedia.org/wiki/����
http://ru.wikipedia.org/wiki/����������
http://ru.wikipedia.org/wiki/����������_(�����)
http://ru.wikipedia.org/wiki/�������_��_�����
http://ru.wikipedia.org/wiki/���������_������
http://ru.wikipedia.org/wiki/����������
http://ru.wikipedia.org/wiki/�������_�������
http://ru.wikipedia.org/wiki/�����
http://ru.wikipedia.org/wiki/�����_(������)
http://ru.wikipedia.org/wiki/�������_�������
http://ru.wikipedia.org/wiki/�������_������_���
http://ru.wikipedia.org/wiki/�������,_�����
http://ru.wikipedia.org/wiki/������_(������������_�������)
http://ru.wikipedia.org/wiki/�������_(�����_�����������)
http://ru.wikipedia.org/wi

http://ru.wikipedia.org/wiki/Цонев,_Коста_Димитров
http://ru.wikipedia.org/wiki/Чепецкий сайт
http://ru.wikipedia.org/wiki/Черепанов,_Иван_Иванович
http://ru.wikipedia.org/wiki/Чжан_Цзе?previous=yes
http://ru.wikipedia.org/wiki/Шаблон:Долина/doc
http://ru.wikipedia.org/wiki/Шаблон:Книга:Киев:_Энциклопедический_справочник/doc
http://ru.wikipedia.org/wiki/Шаблон:Книга:Lacroix-Wells:Japanese_Cruisers_of_the_Pacific_War/doc
http://ru.wikipedia.org/wiki/Шаблон:Медиахолдинг
http://ru.wikipedia.org/wiki/Шаблон:РисПортала/Новейшее_время
http://ru.wikipedia.org/wiki/Шаблон:Спортивная_таблица/Футбол/bolivia/lfpb/20122013/clausura/Дата_обновления
http://ru.wikipedia.org/wiki/Шаблон:Флагификация/Албания
http://ru.wikipedia.org/wiki/Шаблон:Potd/2007-09
http://ru.wikipedia.org/wiki/Школы_Фонтенбло
http://ru.wikipedia.org/wiki/Штаммлер,_Рудольф
http://ru.wikipedia.org/wiki/Эдмунд_Швинглхурст
http://ru.wikipedia.org/wiki/Элеонора_Езерская
http://ru.wikipedia.org/wiki/Эмералд_(тауншип,_Миннесота)
http:

In [73]:
number = re.compile('^/wiki/[^/]+_\(число\)$')

In [74]:
if number.match('/wiki/16497991_(число)'):
    print 'ok'

ok


In [75]:
category = re.compile('^/wiki/Категория:')

In [77]:
if category.match('/wiki/Категория:Персоналии:Луцк'):
    print 'OK'

OK


In [78]:
help(set)

Help on class set in module __builtin__:

class set(object)
 |  set() -> new empty set object
 |  set(iterable) -> new set object
 |  
 |  Build an unordered collection of unique elements.
 |  
 |  Methods defined here:
 |  
 |  __and__(...)
 |      x.__and__(y) <==> x&y
 |  
 |  __cmp__(...)
 |      x.__cmp__(y) <==> cmp(x,y)
 |  
 |  __contains__(...)
 |      x.__contains__(y) <==> y in x.
 |  
 |  __eq__(...)
 |      x.__eq__(y) <==> x==y
 |  
 |  __ge__(...)
 |      x.__ge__(y) <==> x>=y
 |  
 |  __getattribute__(...)
 |      x.__getattribute__('name') <==> x.name
 |  
 |  __gt__(...)
 |      x.__gt__(y) <==> x>y
 |  
 |  __iand__(...)
 |      x.__iand__(y) <==> x&=y
 |  
 |  __init__(...)
 |      x.__init__(...) initializes x; see help(type(x)) for signature
 |  
 |  __ior__(...)
 |      x.__ior__(y) <==> x|=y
 |  
 |  __isub__(...)
 |      x.__isub__(y) <==> x-=y
 |  
 |  __iter__(...)
 |      x.__iter__() <==> iter(x)
 |  
 |  __ixor__(...)
 |      x.__ixor__(y) <==> x^=y
 |  
 

In [79]:
features_list = list(((set(features_list).discard(wiki_segment)).union(set(all_features).intersection(set(wiki_features.keys()))))

SyntaxError: unexpected EOF while parsing (<ipython-input-79-e4bd08e063bc>, line 1)

In [81]:
pd.Series(data=9, index=np.arange(10))

0    9
1    9
2    9
3    9
4    9
5    9
6    9
7    9
8    9
9    9
dtype: int64

In [82]:
import pandas as pd
s = pd.Series(range(10, 20))
print s.min()
f = 6
print s.append(pd.Series([f])).min()

10
6


In [24]:
date = re.compile('.*/\d\d/\d\d/')
if date.match('/news/2008/10/13/pizza'):
    print 'contains date'

contains date


In [8]:
help(re)

Help on module re:

NAME
    re - Support for regular expressions (RE).

FILE
    /usr/lib/python2.7/re.py

MODULE DOCS
    http://docs.python.org/library/re

DESCRIPTION
    This module provides regular expression matching operations similar to
    those found in Perl.  It supports both 8-bit and Unicode strings; both
    the pattern and the strings being processed can contain null bytes and
    characters outside the US ASCII range.
    
    Regular expressions can contain both special and ordinary characters.
    Most ordinary characters, like "A", "a", or "0", are the simplest
    regular expressions; they simply match themselves.  You can
    concatenate ordinary characters, so last matches the string 'last'.
    
    The special characters are:
        "."      Matches any character except a newline.
        "^"      Matches the start of the string.
        "$"      Matches the end of the string or just before the newline at
                 the end of the string.
        "*"    

In [27]:
talk = re.compile('.+Обсуждение:')
if talk.match('/wiki/Обсуждение:Эль-Пино'):
    print 'contains talk'

contains talk
